# CSE527 Homework 3
**Due date: 23:59 on 4.14, 2024 **

## Google Colab Tutorial
---
Go to https://colab.research.google.com/notebooks/, you will see a tutorial named "Welcome to Colaboratory" file, where you can learn the basics of using google colab.

Settings used for assignments: ***Edit -> Notebook Settings -> Runtime Type (Python 3)***.


## Description
---
This homework contains three parts.  
```
Part 1 - Spinning up yolov5 and yolov9 (10 points)    
          You will be using pretrained models from git repo/libraries to run detection of set of images
Part 2 - Finetuning DETR to detect object centroids (60 points)
          You will complete the 5 missing code blocks in the Detection transformer's training pipeline
Part 3 - Implementing UNet based segmentation model (30 points)
          You will code the slightly modified versions of UNet segmentation models  
```

Note: GPU is necessary to complete parts 2 and 3. So keeping in mind that colab has limited GPU time for users, try to use conserve GPU by disabling it in part 1.







## HW3 Part 1
---
While deep learning-based models are getting more and more complicated, it is relatively easy to run them. In this assignment part, we will try to use YOLOv5 and YOLOv9 models on a small subset of MS COCO images.

**Organizing the homework project files in your working space**
```
{last name}_{first name}_{sbu id}_hw3/
├── part1/
│   └── CSE527_24S_HW3.ipynb
├── part2/
│   ├── CSE527_24S_HW3_P2.ipynb
│   └── detr/
├── part3/
│   ├── CSE527_24S_HW3_P3.ipynb
│   └── unet/
└── coco/
```

Create a directory named  `{last name}_{first name}_{sbu id}_hw3` and this will server as your `HW3_ROOT_PATH`.
Further you will have the 3 parts organized inside it as shown above. The coco dataset we will be using will also be in the root directory.

### MS COCO subset dataset

Through out the homework 3 we will be using a small subset of MS COCO dataset (https://cocodataset.org/#home). Given the limited resources (gpu), you will only work on one class of objects: i.e, planes. Overall there are 500 images for training and 50 images for validation.

To load the dataset, place the [coco dataset](https://drive.google.com/drive/folders/1wPcm6V5kwd_iUzh1pvpZpNn9NRUiSNbZ?usp=sharing) at `HW3_ROOT_PATH`.

Add the COCO folder as shortcut to root hw directory. To do this, open the above link and click on "Add shortcut to Drive" button (drive symbol with a plus) and navigate to correct path and add shortcut.



### Setting path variable

In [5]:
HW3_ROOT_PATH = '/content/gdrive/MyDrive/Colab Notebooks/Debnath_Tanoy_116084356_hw3' #'FILL IN YOUR HOMEWORK3 ROOT HERE'
PATH_TO_PART1 = HW3_ROOT_PATH + '/part1'
PATH_TO_COCO = HW3_ROOT_PATH + '/coco'
# TRY to use these variable wherever you load/save some file

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')
# CD into root of your homework3 directory


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [7]:
%cd  $HW3_ROOT_PATH

/content/gdrive/MyDrive/Colab Notebooks/Debnath_Tanoy_116084356_hw3


## Part 1
---
You will use the models [YOLOv5](https://github.com/ultralytics/yolov5) and [YOLOv9](https://github.com/ultralytics/ultralytics) to detect the objects in the 50 validation images at `PATH_TO_COCO/val2017/`.

In this part, you should use the above mentioned libraries as much as you can.

You should try using the _medium_ sized newtorks for both tasks.

## YOLOv5 task (5 points)
There are 50 images in the directory `PATH_TO_COCO/val2017/`. You should use [YOLOv5](https://github.com/ultralytics/yolov5) to detect as many objects as possible and display those bounding boxes overlaying on the input image.

The 50 predicted images should all be saved to a folder and displayed using the method `show_images(folder_path)`


In [8]:
!mkdir -p '{PATH_TO_PART1}'
%cd $PATH_TO_PART1

/content/gdrive/MyDrive/Colab Notebooks/Debnath_Tanoy_116084356_hw3/part1


In [9]:
import matplotlib.pyplot as plt
import os
import numpy as np
from PIL import Image
import cv2

def show_images(folder_path):
    image_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    image_paths = image_paths[:50]
    rows = 5
    cols = 10
    fig, axes = plt.subplots(rows, cols, figsize=(18, 8))
    for i, image_path in enumerate(image_paths):
        image = cv2.imread(image_path)
        row = int(i / cols)
        col = i % cols
        axes[row, col].imshow(image)
        axes[row, col].set_title(os.path.basename(image_path))
        axes[row, col].axis('off')
    plt.tight_layout()
    plt.show()

In [10]:
show_images('/content/gdrive/MyDrive/Colab Notebooks/Debnath_Tanoy_116084356_hw3/coco/val2017')

### Run pretrained model to generate annotated images

In [12]:
# Get the img locations for all images
imgs = []
for file in os.listdir('/content/gdrive/MyDrive/Colab Notebooks/Debnath_Tanoy_116084356_hw3/coco/val2017'):
    imgs.append(os.path.join('/content/gdrive/MyDrive/Colab Notebooks/Debnath_Tanoy_116084356_hw3/coco/val2017', file))
imgs[:5]

['/content/gdrive/MyDrive/Colab Notebooks/Debnath_Tanoy_116084356_hw3/coco/val2017/000000442480.jpg',
 '/content/gdrive/MyDrive/Colab Notebooks/Debnath_Tanoy_116084356_hw3/coco/val2017/000000199977.jpg',
 '/content/gdrive/MyDrive/Colab Notebooks/Debnath_Tanoy_116084356_hw3/coco/val2017/000000459467.jpg',
 '/content/gdrive/MyDrive/Colab Notebooks/Debnath_Tanoy_116084356_hw3/coco/val2017/000000135673.jpg',
 '/content/gdrive/MyDrive/Colab Notebooks/Debnath_Tanoy_116084356_hw3/coco/val2017/000000383621.jpg']

In [13]:
# Reference: https://pytorch.org/hub/ultralytics_yolov5/


##########--WRITE YOUR CODE HERE--##########
import torch
from PIL import Image

# Load YOLOv5 model from PyTorch Hub
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Get the img locations for all images
imgs = []
for file in os.listdir('/content/gdrive/MyDrive/Colab Notebooks/Debnath_Tanoy_116084356_hw3/coco/val2017'):
    imgs.append(os.path.join('/content/gdrive/MyDrive/Colab Notebooks/Debnath_Tanoy_116084356_hw3/coco/val2017', file))

# Perform object detection on each image
results = []
for img_path in imgs:
    img = Image.open(img_path)
    result = model(img)  # Perform inference
    results.append(result)

# Print and save results
for result in results:
  ##########-------END OF CODE-------##########
    result.print()
    result.save()  # Save results (optional)

/usr/local/lib/python3.10/dist-packages/torch/hub.py:294: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip


requirements: Ultralytics requirements ['gitpython>=3.1.30', 'pillow>=10.3.0'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 221.5 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: Pillow 9.4.0
    Uninstalling Pillow-9.4.0:
      Successfully uninstalled Pillow-9.4.0

requirements: AutoUpdate success ✅ 9.3s, installed 2 packages: ['gitpython>=3.1.30', 'pillow>=10.3.0']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



YOLOv5 🚀 2024-4-15 Python-3.10.12 torch-2.2.1+cu121 CPU

100%|██████████| 14.1M/14.1M [00:00<00:00, 79.0MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
image 1/1: 425x640 2 airplanes, 2 trucks
Speed: 25.4ms pre-process, 974.9ms inference, 42.7ms NMS per image at shape (1, 3, 448, 640)
Saved 1 image to runs/detect/exp
image 1/1: 640x480 1 airplane
Speed: 9.9ms pre-process, 509.9ms inference, 5.7ms NMS per image at shape (1, 3, 640, 480)
Saved 1 image to runs/detect/exp2
image 1/1: 423x640 1 airplane
Speed: 8.8ms pre-process, 544.8ms inference, 4.6ms NMS per image at shape (1, 3, 448, 640)
Saved 1 image to runs/detect/exp3
image 1/1: 427x640 2 airplanes
Speed: 9.4ms pre-process, 475.1ms inference, 6.4ms NMS per image at shape (1, 3, 448, 640)
Saved 1 image to runs/detect/exp4
image 1/1: 427x640 1 airplane
Speed: 9.7ms pre-process, 485.4ms inference, 5.5ms NMS per image at shape (1, 3, 448, 640)
Saved 1 image to runs/

### Viewing the generated images
(WARNING: Before running the code in this section, follow the steps, as there is a bug with show_images function)  
Steps:
- First run the above code as it is to generate annotated images
- Next restart the runtime
- Run code just before the "Run pretrained model to generate annotated images" cell.
- Continue running the code from this cell onwards to view the images

In [14]:
# Images generated at: PATH_TO_PART1/runs/detect/exp/
# Move them from there to directory at PATH_TO_PART1/result
import os

source_all = os.path.join('/content/gdrive/MyDrive/Colab Notebooks/Debnath_Tanoy_116084356_hw3/part1', 'runs')
MODEL_PATH = os.path.join('/content/gdrive/MyDrive/Colab Notebooks/Debnath_Tanoy_116084356_hw3/part1', 'yolov5m.pt')
IMG_PATH = os.path.join(source_all, 'detect', 'exp')


In [15]:
# Cleaning the directory
!rm -rf '{source_all}'
!rm -rf '{MODEL_PATH}'

## YOLOv9 task (5 points)
You will repeat the same process you followed above but this time you will use YOLOv9.

In [16]:
!pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 29.1/107.7 GB disk)


In [17]:
import os
from ultralytics import YOLO
# Get image location
IMG_PATH = os.path.join('/content/gdrive/MyDrive/Colab Notebooks/Debnath_Tanoy_116084356_hw3/coco/val2017')
IMG_PATH

'/content/gdrive/MyDrive/Colab Notebooks/Debnath_Tanoy_116084356_hw3/coco/val2017'

In [22]:
##########-------WRITE YOUR CODE HERE-------##########
!rm -rf runs/detect/predict
## USE THIS SPACE
!yolo predict model=yolov8n.pt source='/content/gdrive/MyDrive/Colab Notebooks/Debnath_Tanoy_116084356_hw3/coco/val2017'
##########-------END OF CODE-------##########

100% 6.23M/6.23M [00:00<00:00, 96.0MB/s]
Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU (Intel Xeon 2.20GHz)
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs

image 1/50 /content/gdrive/MyDrive/Colab Notebooks/Debnath_Tanoy_116084356_hw3/coco/val2017/000000005477.jpg: 352x640 1 airplane, 205.3ms
image 2/50 /content/gdrive/MyDrive/Colab Notebooks/Debnath_Tanoy_116084356_hw3/coco/val2017/000000022396.jpg: 512x640 1 airplane, 222.8ms
image 3/50 /content/gdrive/MyDrive/Colab Notebooks/Debnath_Tanoy_116084356_hw3/coco/val2017/000000052412.jpg: 448x640 4 cars, 1 airplane, 190.8ms
image 4/50 /content/gdrive/MyDrive/Colab Notebooks/Debnath_Tanoy_116084356_hw3/coco/val2017/000000071711.jpg: 448x640 6 persons, 3 airplanes, 164.9ms
image 5/50 /content/gdrive/MyDrive/Colab Notebooks/Debnath_Tanoy_116084356_hw3/coco/val2017/000000084752.jpg: 448x640 7 cars, 1 airplane, 170.9ms
image 6/50 /content/gdrive/MyDrive/Colab Notebooks/Debnath_Tanoy_116084356_

In [23]:
# Images generated at path: PATH_TO_PART1/runs/detect/predict
import os
#MODEL_PATH = os.path.join('/content/gdrive/MyDrive/Colab Notebooks/Debnath_Tanoy_116084356_hw3/part1', 'yolov9m.pt')
RES_PATH = os.path.join('/content/gdrive/MyDrive/Colab Notebooks/Debnath_Tanoy_116084356_hw3/part1', 'runs', 'detect', 'predict')
os.makedirs(RES_PATH, exist_ok=True)
# Now RES_PATH should point to the desired directory
show_images(RES_PATH)
#show_images('runs/detect/predict')

## Submission guidelines
---
Plagiarism: plagiarism is strictly forbidden.   
Note: Please be advised that uploading your homework assignments to public platforms, such as GitHub, is strictly prohibited. Sharing your homework solutions in this manner (even after the course completion) constitutes a violation of academic integrity and will be treated as such.

### How to submit part 1
```
├── part1/
│   └── CSE527_24S_HW3.ipynb
```

Part 1 will only have the .ipynb file inside. Make sure all the code cells are executed and the output for each cell is present.


### Overall submission

Before preparing the zip fot BrightSpace submision, you will create _drive_url.txt_. This file will contain the Google drive URL `https://drive.google.com/drive/folders/FOLDER_ID?usp=drive_link` for the root working directory.
Permit the grading TAs to access the folder: **jaqin@cs.stonybrook.edu, rmodhugu@cs.stonybrook.edu**

Note: datasets or learned weights (except heads.pth) should not be a part of brightspace submission.  

This zip file should include the below mentioned files in the same folder structure:
```
{last name}_{first name}_{sbu id}_hw2/
├── part1/
│   └── CSE527_24S_HW3.ipynb
├── part2/
│   ├── CSE527_24S_HW3_P2.ipynb
│   └── detr/
│       ├── checkpoints/
│       │   └── heads.pth
│       ├── datasets/
│       │   ├── coco.py
│       │   ├── coco_eval.py
│       │   └── ...
│       ├── models/
│       │   ├── detr.py
│       │   ├── matcher.py
│       │   ├── ...
│       │   └── ...
│       ├── utils/
│       │   ├── ...
│       │   └── ...
│       ├── engine.py
│       ├── main.py
│       ├── hubconf.py
│       └── requirements.txt
├── part3/
│   ├── CSE527_24S_HW3_P3.ipynb
│   └── unet/
│       ├── utils/
│       │   ├── ...
│       │   └── ...
│       ├── coco.py
│       ├── main.py
│       ├── transforms.py
│       └── unet.py
└── drive_url.txt
```


Other Notes:  
Colab has a good feature of version control, you should take advantage of this to save your work properly. However, the timestamp of the submission made in brightspace is the only one that we consider for grading. To be more specific, we will only grade the version of your code right before the timestamp of the submission made in brightspace.

You are encouraged to post and answer questions on Piazza. Based on the amount of email that we have received in past years, there might be dealys in replying to personal emails. Please ask questions on Piazza and send emails only for personal issues.

**Late submission penalty:** <br>
There will be a 10% penalty per day for late submission. However, you will have 3 days throughout the whole semester to submit late without penalty. Note that the grace period is calculated by days instead of hours. If you submit the homework one minute after the deadline, one late day will be counted. Likewise, if you submit one minute after the deadline, the 10% penaly will be imposed if not using the grace period.